In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
cap = cv2.VideoCapture('../1/vtest.avi')
if not cap.isOpened():
    print('Error~')
    
height, width = (int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))

In [3]:
TH = 80# insentity threshold
AREA_TH = 100# area threshold
mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
acc_bgr = np.zeros(shape=(height, width, 3), dtype=np.float32)

In [4]:
t = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    t += 1
    blur = cv2.GaussianBlur(frame, (5, 5), .0)
    
    if t < 50:
        cv2.accumulate(blur, acc_bgr)
        continue
        
    elif t == 50:
        bkg_bgr = acc_bgr / t
        
    diff_bgr = np.uint8(cv2.absdiff(np.float32(blur), bkg_bgr))
    db, dg, dr = cv2.split(diff_bgr)
    ret, bb = cv2.threshold(db, TH, 255, cv2.THRESH_BINARY)
    ret, bg = cv2.threshold(dg, TH, 255, cv2.THRESH_BINARY)
    ret, br = cv2.threshold(dr, TH, 255, cv2.THRESH_BINARY)
    
    bimage = cv2.bitwise_or(bb, bg)
    bimage = cv2.bitwise_or(br, bimage)
    bimage = cv2.erode(bimage, None, 5)
    bimage = cv2.dilate(bimage, None, 5)
    bimage = cv2.erode(bimage, None, 7)
    
    contours, hierarcy = cv2.findContours(bimage, mode, method)
    cv2.drawContours(frame, contours, -1, (255, 0, 0), 1)
    for i, cnt in enumerate(contours):
        area = cv2.contourArea(cnt)
        if area > AREA_TH:
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
            
    cv2.imshow('frame', frame)
    cv2.imshow('bimage', bimage)
    cv2.imshow('diff_bgr', diff_bgr)
    
    key = cv2.waitKey(20)
    if key ==  27:
        break
    
if cap.isOpened():
    cap.release()

cv2.destroyAllWindows()